In [1]:
from astropy.io import fits
import sys
import pandas as pd
from astropy.table import unique, Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc, sky_match
from numcosmo_py.external.pyssc import pyssc as PySSC
from numcosmo_py.helper import npa_to_seq
import timeit
Ncm.cfg_init()

from getdist.mcsamples import  MCSamples
from getdist import plots

import getdist
sys.path.insert(0, "/global/homes/l/lettieri/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "/global/homes/l/lettieri/gcr-catalogs")
import GCRCatalogs
GCRCatalogs.set_root_dir_by_site("nersc")
from GCR import GCRQuery
table_redmapper_halos = fits.open("catalogs/redmapper_halos.fits")[1].data

#%matplotlib inline

In [2]:
pz_min = 0.2
pz_max = 1.0
logRmin = 0
logRmax = 3 

# We are loading the cosmoDC2_redmapper catalalog with richness
redmapper_cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_redmapper_v0.8.1")

# Taking some important information about the fiducial cosmological model and survey region
redmapper_cosmology = redmapper_cat.cosmology
sky_area = redmapper_cat.sky_area

# Listing all quantities in the choosen catalog
#np.sort(redmapper_cat.list_all_quantities())
#print(np.sort(redmapper_cat.list_all_quantities()))



#image_cat['halo_mass'].size()

data_redmapper = redmapper_cat.get_quantities(
    ["cluster_id", "redshift", "redshift_err", "richness", "richness_err", "ra", "dec"],
    filters=[
        f"redshift > {pz_min}",
        f"redshift < {pz_max}",
        f"richness > {10**logRmin}",
        f"richness < {10**logRmax}",
    ],
)
print("ra interval")
print(data_redmapper['ra'].min())
print(data_redmapper['ra'].max())
print('\n')

print("dec interval")
print(data_redmapper['dec'].min())
print(data_redmapper['dec'].max())
print('\n')

print("z interval")
print(data_redmapper['redshift'].min())
print(data_redmapper['redshift'].max())
print('\n')

print("richness interval")
print(data_redmapper['richness'].min())
print(data_redmapper['richness'].max())
print('\n')

print("sky_area = %.3f sqd" % (sky_area))
data_redmapper = Table(data_redmapper)
data_redmapper['richness'] = np.log(data_redmapper['richness'])


cut = data_redmapper['richness'].min()
lnR_max = data_redmapper['richness'].max()

ra interval
47.83447574575201
75.9017151136021


dec interval
-46.49188930864035
-24.68019523136432


z interval
0.20012587
0.9999194


richness interval
5.007845
266.29504


sky_area = 439.790 sqd


In [3]:
plt.hist(data_redmapper['richness'],bins=1000)
plt.xscale("log")
plt.yscale("log")
plt.ylabel("Number of detections")
plt.xlabel(r"$\ln \lambda$")
plt.title("Redmapper detections as a function of richness")
plt.show()

plt.hist(data_redmapper['redshift'],bins=1000)
plt.title("Redmapper detections as a function of z")
plt.ylabel("Number of detections")
plt.xlabel(r"$z_{ph}$")
plt.show()

# Cosmological Model

In [4]:
#cosmological model
cosmo = Nc.HICosmoDEXcdm()
#cosmo.omega_x2omega_k()
#cosmo.param_set_by_name("Omegak", 0.00)
cosmo.param_set_by_name("Omegax", redmapper_cosmology.Ode0)
cosmo.param_set_by_name("H0", redmapper_cosmology.H0.value)
cosmo.param_set_by_name("Omegab", redmapper_cosmology.Ob0)
cosmo.param_set_by_name("Omegac", redmapper_cosmology.Odm0)  # 0.2603
cosmo.param_set_by_name("w", -1.0)  # -1.0

prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA", 3.0)
prim.props.n_SA = redmapper_cosmology.n_s
reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

#matter power spectrum
tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()
psf.prepare(cosmo)
print(cosmo.sigma8(psf))

old_amplitude = np.exp(prim.props.ln10e10ASA)
prim.props.ln10e10ASA = np.log(
    (redmapper_cosmology.sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude
)
print(redmapper_cosmology.sigma8, cosmo.sigma8(psf) , prim.props.ln10e10ASA)



dist = Nc.Distance.new(2.0)
dist.prepare(cosmo)

#parameters to be fitted

cosmo.set_property("Omegac_fit", True)
#cosmo.set_property("w_fit", True)
prim.set_property("ln10e10ASA_fit", True)


0.7764593327536546
0.8 0.8 3.0597349179015914


# Cluster Mass-Richness and photoz model 

In [5]:
cut = np.log(20)
cluster_m = Nc.ClusterMassAscaso(lnRichness_min = cut ,lnRichness_max = lnR_max)
cluster_m.param_set_by_name("mup0", 4.11945317984568)
cluster_m.param_set_by_name("mup1", 1.11236863168254)
cluster_m.param_set_by_name("mup2", 0.299464198653602)
cluster_m.param_set_by_name("sigmap0", 0.471246505057074)
cluster_m.param_set_by_name("sigmap1", -0.0591674971985774)
cluster_m.param_set_by_name("sigmap2", -0.109077938873429)
cluster_m.lnM0 = np.log(10**(14.3))
cluster_m.z0 = 0.5

'''
cluster_m.param_set_by_name("mup0", 3.115)
cluster_m.param_set_by_name("mup1", 1.192)
cluster_m.param_set_by_name("mup2", -0.9802 )
cluster_m.param_set_by_name("sigmap0", 0.7078 )
cluster_m.param_set_by_name("sigmap1", -0.2283)
cluster_m.param_set_by_name("sigmap2", 0.1264)
'''
                         
cluster_m.param_set_by_name("cut", cut)
print(cluster_m.n_limits(cosmo)/np.log(10))
print(cluster_m.p_limits(cosmo, [cut] , [0])/np.log(10))

'''
cluster_m.set_property("mup0_fit", True)
cluster_m.set_property("mup1_fit", True)
cluster_m.set_property("mup2_fit", True)
cluster_m.set_property("sigmap0_fit", True)
cluster_m.set_property("sigmap1_fit", True)
cluster_m.set_property("sigmap2_fit", True)
'''



#cluster_z = Nc.ClusterPhotozGauss(pz_min = pz_min,pz_max = pz_max)
cluster_z = Nc.ClusterRedshiftNodist(z_min = pz_min,z_max = pz_max)

[12. 16.]
[12. 16.]


# Cluster abundance model

In [6]:
mulf = Nc.MultiplicityFuncDespali.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.VIRIAL)
#mulf.set_Delta(200.0)

# Discutivel
mulf_tinker = Nc.MultiplicityFuncTinker.new()
mulf_tinker.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf_tinker.set_Delta(200.0)
hmf_tinker = Nc.HaloMassFunction.new(dist, psf, mulf_tinker)
hmf_tinker.prepare(cosmo)
hmf_tinker.set_area(sky_area * (np.pi / 180) ** 2)
hbias = Nc.HaloBiasTinker.new(hmf_tinker)


hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.prepare(cosmo)
hmf.set_area(sky_area * (np.pi / 180) ** 2)
#hbias = Nc.HaloBiasDespali.new(hmf_tinker)
cad = Nc.ClusterAbundance.new(hmf, hbias)
cad.set_area(sky_area * (np.pi / 180) ** 2)


cad.prepare(cosmo , cluster_z , cluster_m)
print(cad.n(cosmo ,cluster_z , cluster_m) * 1/0.9)
red_mapper_20 = data_redmapper[(data_redmapper['richness']>= np.log(20)) & (data_redmapper['redshift'] >= pz_min) & (data_redmapper['redshift'] <= pz_max)]
print(len(red_mapper_20['richness']))

nbins=100
lnM_obs = np.linspace(cut, lnR_max , nbins +1)
lnM_obs_bin = [(lnM_obs[i] +lnM_obs[i+1])/2 for i in range(len(lnM_obs)-1)]
z_obs = np.linspace(pz_min, pz_max , nbins +1)
z_obs_bin = [(z_obs[i] +z_obs[i+1])/2 for i in range(len(z_obs)-1)]
abundance_lnM = [0.8/0.9 * cad.intp_bin_d2n(cosmo, cluster_z, cluster_m, [lnM_obs[i]] , [lnM_obs[i+1]], None, [pz_min] , [pz_max] , [0.00,0.03])  for i in range(len(lnM_obs)-1)]
abundance_z = [0.8/0.9 * cad.intp_bin_d2n(cosmo, cluster_z, cluster_m, [cut] , [lnR_max], None, [z_obs[i]] , [z_obs[i+1]] , [0.00,0.03 * (1 + z_obs_bin[i])])  for i in range(len(z_obs)-1)]

plt.hist(red_mapper_20['richness'],bins=lnM_obs)
plt.plot(lnM_obs_bin,abundance_lnM)
#plt.xscale("log")
plt.yscale("log")
plt.ylabel("Number of detections")
plt.xlabel(r"$\ln \lambda$")
plt.title("Redmapper detections as a function of richness")
plt.show()

plt.hist(red_mapper_20['redshift'],bins=z_obs)
plt.plot(z_obs_bin,abundance_z)
#plt.xscale("log")
plt.yscale("log")
plt.ylabel("Number of detections")
plt.xlabel(r"$z_{ph}$")
plt.title("Redmapper detections as a function of photometric redshift")
plt.show()

#Mset Preparation
mset = Ncm.MSet.new_array([cluster_m, cosmo, cluster_z])
mfunc_oa = Ncm.ObjArray.new()
mfunc_sigma8 = Ncm.MSetFuncList.new("NcHICosmo:sigma8", psf)
mfunc_oa.add(mfunc_sigma8)
mset.prepare_fparam_map()
dset = Ncm.Dataset.new()

4424.560831878961
3662


# Cluster counts binned likelihood

In [7]:
lnR_bins_knots = np.log(np.array([20, 35, 70, 100, 200]))
z_bins_knots = np.array([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0])


kernel_z = np.linspace(0.0, 1.9, num=400 + 1)[1:]
kernels_T = np.zeros((len(z_bins_knots)-1, 400))

S_fullsky_array = PySSC.Sij(kernel_z, kernels_T, cosmo)
S_fullsky = Ncm.Matrix.new_array(S_fullsky_array.flatten(), S_fullsky_array.shape[1])

bin_counts = []
for i in range(len(z_bins_knots)-1):
    bins_coutns_z_i = red_mapper_20[(red_mapper_20['redshift'] >= z_bins_knots[i]) * (red_mapper_20['redshift'] <= z_bins_knots[i+1])]

    for j in range(len(lnR_bins_knots)-1):
        bin_counts.append(0.9/0.8 * len(bins_coutns_z_i[(bins_coutns_z_i['richness'] >= lnR_bins_knots[j]) * (bins_coutns_z_i['richness'] <= lnR_bins_knots[j+1]
                                                                                                     )]))

z_bins_vec = Ncm.Vector.new_array(npa_to_seq(z_bins_knots))
lnR_bins_vec = Ncm.Vector.new_array(npa_to_seq(lnR_bins_knots))

nc_data_binned = Nc.DataClusterNCountsGauss.new(cad)
nc_data_binned.set_size((z_bins_vec.len() - 1) * (lnR_bins_vec.len() - 1))
nc_data_binned.set_init(True)
nc_data_binned.use_norma(True)
nc_data_binned.set_z_obs(z_bins_vec)
nc_data_binned.set_lnM_obs(lnR_bins_vec)
nc_data_binned.set_s_matrix(S_fullsky)
nc_data_binned.peek_mean().set_array(npa_to_seq(bin_counts))
nc_data_binned.peek_mean().log_vals("mean:" , "%.3f" , True)

dset.append_data(nc_data_binned)
# Likelihood
lh = Ncm.Likelihood.new(dset)
color_list = ["blue" , "black", "red" , "green" , "yellow", "brown" , "purple"]
for i in range(len(z_bins_knots)-1):
    abundance_lnR = [cad.intp_bin_d2n(cosmo, cluster_z, cluster_m, [lnR_bins_knots[j]] , [lnR_bins_knots[j+1]], None, [z_bins_knots[i]] , [z_bins_knots[i+1]] , [0.00,0.01]) for j in range(len(lnR_bins_knots)-1)]
    lnR_bins = [(lnR_bins_knots[j] + lnR_bins_knots[j+1])/2 for j in range(len(lnR_bins_knots)-1)]
    bin_counts_z = [bin_counts[ i * (len(lnR_bins_knots)-1) + j] for j in range(len(lnR_bins_knots)-1)]
    err_bin_counts_z = [np.sqrt(bin_counts[ i * (len(lnR_bins_knots)-1) + j]) for j in range(len(lnR_bins_knots)-1)]

    plt.errorbar(lnR_bins, bin_counts_z, yerr=err_bin_counts_z,label=r"$z \in [%.1f,%.1f]$" % (z_bins_knots[i] , z_bins_knots[i+1]),fmt='.' ,color =color_list[i])
    plt.plot(lnR_bins,abundance_lnR,color =color_list[i])





    
#plt.xscale("log")
plt.yscale("log")
plt.ylabel("Number of detections")
plt.xlabel(r"$\ln \lambda$")
plt.title("Redmapper detections as a function of richness")
plt.grid()
#plt.legend()
plt.show()

mean:159.750 82.125 15.750 6.750 286.875 117.000 14.625 6.750 343.125 154.125 19.125 19.125 407.250 137.250 21.375 13.500 424.125 172.125 34.875 15.750 351.000 156.375 23.625 7.875 734.625 324.000 36.000 24.750


/global/u2/l/lettieri/NumCosmo/numcosmo_py/external/pyssc/pyssc.py:170: RuntimeWarning: invalid value encountered in divide
  U1 = Uarr[ibin, :] / Inorm[ibin]
/global/u2/l/lettieri/NumCosmo/numcosmo_py/external/pyssc/pyssc.py:172: RuntimeWarning: invalid value encountered in divide
  U2 = Uarr[jbin, :] / Inorm[jbin]


# Fit without w

In [8]:
#Mset Preparation
mset = Ncm.MSet.new_array([cluster_m, cosmo, cluster_z])
mfunc_oa = Ncm.ObjArray.new()
mfunc_sigma8 = Ncm.MSetFuncList.new("NcHICosmo:sigma8", psf)
mfunc_oa.add(mfunc_sigma8)
mset.prepare_fparam_map()
dset = Ncm.Dataset.new()

fit = Ncm.Fit.factory(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.log_info()
fit.run(Ncm.FitRunMsgs.SIMPLE)
fit.log_info()
fit.obs_fisher()
fit.log_covar()

#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.22                [FREE]
#   -     Omegax[02]:  0.7352              [FIXED]
#   -    Tgamma0[03]:  2.7245              [FIXED]
#   -         Yp[04]:  0.24                [FIXED]
#   -       ENnu[05]:  3.046               [FIXED]
#   -     Omegab[06]:  0.0448              [FIXED]
#   -          w[07]: -1                   [FIXED]
#----------------------------------------------------------------------------------
# Model[05000]:
#   - NcHIPrim : Power Law model for primordial spectra
#------------------------------------------------------------------

# MCMC without w

In [ ]:
cosmo.set_property("w_fit", False)
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new_funcs_array(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL,mfunc_oa)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("mcmc_no_w.fits")

esmcmc.start_run()
esmcmc.run(400)
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# Data used:
#   - NcDataClusterNCountsGauss
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Model set:
#----------------------------------------------------------------------------------
# Model[03000]:
#   - NcHICosmo : XCDM - Constant EOS
#----------------------------------------------------------------------------------
# Model parameters
#   -         H0[00]:  71                  [FIXED]
#   -     Omegac[01]:  0.216863392


(process:1993241): NUMCOSMO-WARNING **: 11:54:14.442: ncm_integrate_2dim: number of evaluations 10000055 >= maximum number of evaluations 10000000 (nregions 76924, fail 1, result   8.06132908102114e-13, error   1.16620599369764e-19).


(process:1993241): NUMCOSMO-WARNING **: 11:54:22.035: ncm_integrate_2dim: number of evaluations 10000055 >= maximum number of evaluations 10000000 (nregions 76924, fail 1, result    2.2518160509368e-13, error   5.37485751646438e-20).


(process:1993241): NUMCOSMO-WARNING **: 11:54:25.889: ncm_integrate_2dim: number of evaluations 10000055 >= maximum number of evaluations 10000000 (nregions 76924, fail 1, result   1.45705458272976e-12, error   2.35240627614894e-19).


(process:1993241): NUMCOSMO-WARNING **: 11:54:29.668: ncm_integrate_2dim: number of evaluations 10000055 >= maximum number of evaluations 10000000 (nregions 76924, fail 1, result   5.66265298515923e-14, error   2.18741395809577e-20).


(process:1993241): NUMCOSMO-WARNING **: 11:54:33.569: n

#  Elapsed time: 00 days, 00:00:15.2423260
#  iteration            [000000]
#  function evaluations [000151]
#  gradient evaluations [000000]
#  degrees of freedom   [000026]
#  m2lnL     =     244.153306912769 (     223.46066 )
#  Fit parameters:
#     0.218528214997599      3.10225613471083      
# NcmMSetCatalog: Current mean:   2572.6       0.82212      0.21448      3.1603     
# NcmMSetCatalog: Current msd:    452.38       0.00092322   0.0001772    0.0025848  
# NcmMSetCatalog: Current sd:     46356        0.094602     0.018157     0.26486    
# NcmMSetCatalog: Current var:    2.1488e+09   0.0089495    0.00032969   0.070152   
# NcmMSetCatalog: Current tau:    1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99901      1.0059       1.2932       1.1372     
# NcmMSetCatalog: Maximal Shrink factor = 1.29364               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant 

# Fit with w

In [ ]:
#Mset Preparation
cosmo.set_property("w_fit", True)
mset= Ncm.MSet.new_array([cluster_m, cosmo, cluster_z])
mfunc_oa = Ncm.ObjArray.new()
mfunc_sigma8 = Ncm.MSetFuncList.new("NcHICosmo:sigma8", psf)
mfunc_oa.add(mfunc_sigma8)
mset.prepare_fparam_map()
dset= Ncm.Dataset.new()

cosmo.set_property("w_fit", True)
fit = Ncm.Fit.factory(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.log_info()
fit.run(Ncm.FitRunMsgs.SIMPLE)
fit.log_info()
fit.obs_fisher()
fit.log_covar()

# MCMC with w

In [ ]:
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("mcmc_w.fits")

esmcmc.start_run()
esmcmc.run(400)
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()

# Cluster counts unbinned likelihood

In [ ]:
ncdata = Nc.DataClusterNCount.new(cad, "NcClusterPhotozGauss", "NcClusterMassAscaso")
lnM_obs = Ncm.Matrix.new(len(red_mapper_20['richness']),1)
z_obs   = Ncm.Matrix.new(len(red_mapper_20['redshift']),1)
z_obs_params  = Ncm.Matrix.new(len(red_mapper_20['redshift_err']),2)                        

for i in range(len(red_mapper_20['richness'])):
    lnM_obs.set(i,0, np.log(red_mapper_20['richness'][i]))
    z_obs.set(i,0,red_mapper_20['redshift'][i])
    for j in range(0, 2):
        if j == 1:
            z_obs_params.set(i, j, red_mapper_20['redshift_err'][i])
        if j == 0:
            z_obs_params.set(i, j, 0.0)

ncdata.set_lnM_obs(lnM_obs)
ncdata.set_z_obs(z_obs)
ncdata.set_z_obs_params(z_obs_params)
ncdata.true_data(False)
ncdata.set_init(True)

dset.append_data(ncdata)
# Likelihood
lh = Ncm.Likelihood.new(dset)

# Fit

In [ ]:
fit = Ncm.Fit.factory(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_CENTRAL
)

fit.log_info()
fit.run(Ncm.FitRunMsgs.SIMPLE)
fit.log_info()
fit.obs_fisher()
fit.log_covar()

In [ ]:
nwalkers = 500

Ncm.func_eval_set_max_threads(2)
Ncm.func_eval_log_pool_stats()

init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)

apes = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())

esmcmc = Ncm.FitESMCMC.new(fit, nwalkers, init_sampler, apes, Ncm.FitRunMsgs.FULL)
esmcmc.set_nthreads(2)
esmcmc.set_data_file("mcmc_self_calibration_cut_20.fits")

esmcmc.start_run()
esmcmc.run(400)
esmcmc.end_run()

esmcmc.mean_covar()
fit.log_covar()